In [34]:
import pandas as pd
import numpy as np
import scipy.sparse as sps
from scipy import sparse
from sklearn.cluster import KMeans
from numpy import linalg
from scipy.linalg import norm
import math
import matplotlib.pyplot as plt
import time
import seaborn as sns
from matplotlib import colors

In [35]:
def heatmap(DataFrame, cmap_ , color_pallete_label, x_label, y_label, Title, size):
    plt.figure(figsize= size)
    plt.suptitle(Title)
    ax = sns.heatmap(DataFrame ,  annot=True , cmap = cmap_, cbar_kws= {'label': color_pallete_label}
        )
    ax.set(xlabel= x_label, ylabel= y_label)
    plt.show()
def heatmap_noannotation(DataFrame, cmap_ , color_pallete_label, x_label, y_label, Title, size):
    plt.figure(figsize= size)
    plt.suptitle(Title)
    ax = sns.heatmap(DataFrame ,  annot=False , cmap = cmap_, cbar_kws= {'label': color_pallete_label}
        )
    ax.set(xlabel= x_label, ylabel= y_label)
    plt.show()
    
def Custom_Visualization(DataFrame, cmap , color_pallete_label, x_label, y_label, Title, size, bounds):
    plt.figure(figsize=size)
    plt.suptitle(Title)
    norm = colors.BoundaryNorm(bounds, cmap.N)
    ax = sns.heatmap(DataFrame ,  annot=False, cmap = cmap, norm=norm, cbar_kws={'label': color_pallete_label}
        )
    ax.set(xlabel= x_label, ylabel= y_label)
    plt.show()
    
def remove_tick_labels(ax):
    labels = [item.get_text() for item in ax.get_xticklabels()]
    empty_string_labels = ['']*len(labels)
    ax.set_xticklabels(empty_string_labels)
    labels = [item.get_text() for item in ax.get_yticklabels()]
    empty_string_labels = ['']*len(labels)
    ax.set_yticklabels(empty_string_labels)


In [36]:
def initialize_OD (Sample_OD_link):
    Sample_OD = pd.read_csv(Sample_OD_link, index_col = 0, delimiter = '\t')
    Sample_OD.index = Sample_OD.index.map(str)
    for col in Sample_OD.columns:
        Sample_OD[col].values[:] = 0
    return (Sample_OD)

In [66]:
import numpy as np
import scipy.sparse as sp
import cupy as cp

def calculate_rmse(mat1, mat2):
    # Ensure matrices have the same shape
    if mat1.shape != mat2.shape:
        raise ValueError("Matrices must have the same shape")

    # Calculate element-wise squared difference
    squared_diff = np.power(mat1 - mat2, 2) 

    # Calculate overall RMSE
    overall_rmse = cp.sqrt(squared_diff.sum() / (mat1.nnz + mat2.nnz))
    
    # Calculate RMSE of non-zero entries in mat1
    non_zero_indices = mat1.nonzero()
    non_zero_squared_diff = squared_diff[non_zero_indices]
    non_zero_rmse = cp.sqrt(non_zero_squared_diff.sum() / mat1.nnz)
    
    return round(overall_rmse.item(), 4), round(non_zero_rmse.item(), 4)

def calculate_rmse(mat1, mat2):
    # Ensure matrices have the same shape

    # Calculate element-wise squared difference
    squared_diff = np.power(mat1 - mat2, 2) 

    # Calculate overall RMSE
    overall_rmse = cp.sqrt(squared_diff.sum() / (squared_diff.shape[0] * squared_diff.shape[1]) )
    
    return round(overall_rmse.item(), 4)

# Import Matrices

In [67]:
dir = r'C:\Users\n10680535\Queensland University of Technology\Shubham Sharma - Shubham Staff Account\Paper-Last\P8 processed'
dir1 = r'C:\Users\n10680535\Queensland University of Technology\Shubham Sharma - Shubham Staff Account\Paper-Last\P5 processed'
import os

In [68]:
shuffled_ids = pd.read_csv(r'C:\Users\n10680535\Queensland University of Technology\Shubham Sharma - Shubham Staff Account\Paper-Last\Final Analysis\shuffledids.csv')['0'].values
shuffled_ids

array([106, 167,  72,  92, 197,  96, 116,  94,  61,  24, 199,  21,  66,
       170,  47, 123, 162,   3,  15, 134, 156,  54,  49,  57,  13, 186,
       147,  19,  76,  77, 117,  68,  81,  56,  36, 183,  88, 101, 153,
       193, 159,  38,   1,  87,  53,   7, 169,  41, 198, 104,  55,  23,
       176, 120,  93, 190,  51,  45, 179, 107, 137,  14, 142,  60, 164,
        97, 158,  67,  22,   4, 182,   0,  16,  83, 131,  12,  37, 145,
       115, 174, 126, 140, 168, 141, 161,  86, 108,  70,  91, 185, 110,
        80, 122,  27, 127, 177,  73,  74,  84, 154,  20, 125, 111,  42,
       112, 135, 144, 165, 184,  10, 139,  44, 150,   8, 105,  29, 191,
        63,  39,  11, 119,  79,  17, 195,  62, 129,  89, 171,  50,  59,
       181, 173,   2, 160, 130, 172, 118,  48,  34,  46, 146,  28, 113,
       143, 157,  98, 192,  26, 133, 103, 149, 151, 194,  69, 187,  65,
       109,  35, 196,  82,  71,  78,   6, 128, 102, 166, 175,  25, 152,
       155,   9,  52, 138,  43,  85,  95,  90, 148,  40, 114,   

In [69]:
training_days = shuffled_ids[0:170]

In [70]:
training_days

array([106, 167,  72,  92, 197,  96, 116,  94,  61,  24, 199,  21,  66,
       170,  47, 123, 162,   3,  15, 134, 156,  54,  49,  57,  13, 186,
       147,  19,  76,  77, 117,  68,  81,  56,  36, 183,  88, 101, 153,
       193, 159,  38,   1,  87,  53,   7, 169,  41, 198, 104,  55,  23,
       176, 120,  93, 190,  51,  45, 179, 107, 137,  14, 142,  60, 164,
        97, 158,  67,  22,   4, 182,   0,  16,  83, 131,  12,  37, 145,
       115, 174, 126, 140, 168, 141, 161,  86, 108,  70,  91, 185, 110,
        80, 122,  27, 127, 177,  73,  74,  84, 154,  20, 125, 111,  42,
       112, 135, 144, 165, 184,  10, 139,  44, 150,   8, 105,  29, 191,
        63,  39,  11, 119,  79,  17, 195,  62, 129,  89, 171,  50,  59,
       181, 173,   2, 160, 130, 172, 118,  48,  34,  46, 146,  28, 113,
       143, 157,  98, 192,  26, 133, 103, 149, 151, 194,  69, 187,  65,
       109,  35, 196,  82,  71,  78,   6, 128, 102, 166, 175,  25, 152,
       155], dtype=int64)

In [71]:
testing_days = shuffled_ids[170:200]

In [97]:
X1 = pd.read_csv(dir + os.sep + "OD8.csv", index_col = [0,1], header = [0]).astype('float32')
#X2 = pd.read_csv(dir1 + os.sep + "OD5.csv", index_col = [0,1], header = [0]).astype('float32')
#X2.index = pd.MultiIndex.from_tuples([(x + 100, y) for x, y in X2.index])
#X1 = pd.concat([X1, X2])
X1 = X1.replace(np.nan, 0)
non_zero_columns = X1.columns[X1.ne(0).any()]
X1 = X1[non_zero_columns]
#X1.index.name = ['Sample', 'period']
new_index = X1.copy(deep= True).index.set_names(['Sample', 'period'])
X1.index = new_index
unique_ids = X1.index.get_level_values('Sample').unique()
#shuffled_ids = np.random.permutation(unique_ids)
#pd.DataFrame(shuffled_ids).to_csv('shuffledids.csv')
# Create a new MultiIndex with the shuffled values
#new_index = X1.index.set_levels(shuffled_ids, level='Sample')
X1.index = new_index
X1

16675991_16676063  16675991_16676126  16675991_16676131  \
Sample period                                                            
0      1                     0.0                0.0                0.0   
       2                     0.0                0.0                0.0   
       3                     0.0                0.0                0.0   
       4                     0.0                0.0                0.0   
       5                     0.0                0.0                0.0   
...                          ...                ...                ...   
99     92                    0.0                0.0                0.0   
       93                    0.0                0.0                0.0   
       94                    0.0                0.0                0.0   
       95                    0.0                0.0                0.0   
       96                    0.0                0.0                0.0   

               16675991_16676179  16675991_16676307  16675991_16676575  \
Sample period                                                            
0      1                     0.0                0.0                0.0   
       2                     0.0                0.0                0.0   
       3                     0.0                0.0                0.0   
       4                     0.0                0.0                0.0   
       5                     0.0                0.0                0.0   
...                          ...                ...                ...   
99     92                    0.0                0.0                0.0   
       93                    0.0                0.0                0.0   
       94                    0.0                0.0                0.0   
       95                    0.0                0.0                0.0   
       96                    0.0                0.0                0.0   

               16675991_16676578  16675991_16676581  16675991_16676600  \
Sample period                                                            
0      1                     0.0                0.0                0.0   
       2                     0.0                0.0                0.0   
       3                     0.0                0.0                0.0   
       4                     0.0                0.0                0.0   
       5                     0.0                0.0                0.0   
...                          ...                ...                ...   
99     92                    0.0                0.0                0.0   
       93                    0.0                0.0                0.0   
       94                    0.0                0.0                0.0   
       95                    0.0                0.0                0.0   
       96                    0.0                0.0                0.0   

               16675991_16676625  ...  16676628_16676116  16676628_16676126  \
Sample period                     ...                                         
0      1                     0.0  ...                0.0                0.0   
       2                     0.0  ...                0.0                0.0   
       3                     0.0  ...                0.0                0.0   
       4                     0.0  ...                0.0                0.0   
       5                     0.0  ...                0.0                0.0   
...                          ...  ...                ...                ...   
99     92                    0.0  ...                0.0                0.0   
       93                    0.0  ...                0.0                0.0   
       94                    0.0  ...                0.0                0.0   
       95                    0.0  ...                0.0                0.0   
       96                    0.0  ...                0.0                0.0   

               16676628_16676131  16676628_16676156  16676628_16676575  \
Sample period           

In [98]:
X1_training = X1[X1.index.get_level_values(0).isin(training_days)]
X1_testing = X1[X1.index.get_level_values(0).isin(testing_days)]

In [99]:
Y1 = pd.read_csv(dir + os.sep + r"Volume_P8.csv", index_col = [0,1]).astype('float64')
#Y1_2 = pd.read_csv(dir1 + os.sep + r"Volume_P5.csv", index_col = [0,1]).astype('float64')
#Y1_2.index = pd.MultiIndex.from_tuples([(x + 100, y) for x, y in Y1_2.index])
#Y1 = pd.concat([Y1_2, Y1])
Y1 = Y1.replace(np.nan, 0)
non_zero_columns = Y1.columns[Y1.ne(0).any()]
Y1 = Y1[non_zero_columns]
new_index = Y1.copy(deep= True).index.set_names(['Sample', 'period'])
Y1.index = new_index
Y1 = Y1
Y1 = Y1.reindex(X1.index)
Y1 = Y1.replace(np.nan, 0)


In [100]:
Y2 = pd.read_csv(dir + os.sep + r"Speed_P8.csv", index_col = [0,1]).astype('float64')
Y2_2 = pd.read_csv(dir1 + os.sep + r"Speed_P5.csv", index_col = [0,1]).astype('float64')
Y2_2.index = pd.MultiIndex.from_tuples([(x + 100, y) for x, y in Y1_2.index])
Y2 = pd.concat([Y1_2, Y2])
non_zero_columns = Y2.columns[Y2.ne(0).any()]
Y2 = Y2[Y1.columns]
new_index = Y2.copy(deep= True).index.set_names(['Sample', 'period'])
Y2.index = new_index
Y2 = Y2.reindex(X1.index)
Y2 = Y2.replace(np.nan, 0)

In [101]:
Y1_training = Y1[Y1.index.get_level_values(0).isin(training_days)]
Y1_testing = Y1[Y1.index.get_level_values(0).isin(testing_days)]
Y2_training = Y2[Y2.index.get_level_values(0).isin(training_days)]
Y2_testing = Y2[Y2.index.get_level_values(0).isin(testing_days)]

In [103]:
R_fxy = pd.read_csv(dir + os.sep + "Assignment8.csv", index_col = 0, header = [0]).astype('float64')
R_fxy2 = pd.read_csv(dir1 + os.sep + "Assignment5.csv", index_col = 0, header = [0]).astype('float64')
R_fxy = R_fxy.div( R_fxy.sum(axis = 1), axis = 0)
R_fxy = R_fxy.replace(np.nan, 0)
R_fxy = R_fxy.replace(np.inf, 0)
for column in Y1.columns:
    # Check if the column exists in the DataFrame
    if column not in R_fxy.columns:
        # Add the column and fill with 0
        R_fxy[column] = 0
R_fxy = R_fxy.T
for column in X1.columns:
    # Check if the column exists in the DataFrame
    if column not in R_fxy.columns:
        # Add the column and fill with 0
        R_fxy[column] = 0
R_fxy = R_fxy.T
R_fxy = (R_fxy+R_fxy2).replace(np.nan, 0)/2
R_fxy = R_fxy[Y1.columns].T
R_fxy = R_fxy[X1.columns].T
R_fxy = R_fxy.replace(np.nan, 0)
R_fxy = R_fxy.replace(np.inf, 0)

In [104]:
common_ODs = R_fxy.index.to_list()
common_links = R_fxy.columns.to_list()
X1 = X1[common_ODs]
Y1 = Y1[common_links]
Y2 = Y2[common_links]
R_fxy = R_fxy.T
R_fxy = R_fxy[common_ODs].T[common_links].T
R_fxy = R_fxy.replace(np.nan, 0)
R_fxy.sum().sum()

6224.664011244753

# Converting Matrices into Sparse Matrices

X1_scale = pd.DataFrame(X1.max(axis = 0))
Y1_scale = pd.DataFrame(Y1.max(axis = 0))
Y2_scale = pd.DataFrame(Y2.max(axis = 0))

X1_r = X1/X1.max(axis = 0)
Y1_r = Y1/Y1.max(axis = 0)
Y2_r = Y2/Y2.max(axis = 0)

from cupyx.scipy.sparse import diags, kron, csr_matrix
import cupy as cp
X1_cp = csr_matrix(cp.asarray(np.array((X1/X1.max(axis = 0)).T)))
Y1_cp = csr_matrix(cp.asarray(np.array((Y1/Y1.max(axis = 0)).T)))
Y2_cp = csr_matrix(cp.asarray(np.array((Y2/Y2.max(axis = 0)).T)))

R_fxy_cp = csr_matrix(cp.asarray(np.array(R_fxy)))

In [105]:
from cupyx.scipy.sparse import diags, kron, csr_matrix
import cupy as cp
X1_cp = cp.asarray(np.array((X1_training).T))
Y1_cp = cp.asarray(np.array((Y1_training).T))
Y2_cp = cp.asarray(np.array((Y2).T))

R_fxy_cp = cp.asarray(np.array(R_fxy))

In [106]:
R_fxy.sum().sum()

6224.664011244753

In [107]:
Traffic_datasets = [Y1_cp ]#, Y2_cp]
Y_cp = {}
Y_cp = {index: dataset for index, dataset in enumerate(Traffic_datasets)}

In [108]:
R_fxy_cp = R_fxy_cp.T

# Initializing the solution  
1. r_x = Rank of factorization for X
2. r_y = Rank of Factorization for Y

In [109]:
r_x = 40
r_y = 40
U_x =  cp.random.rand(X1_cp.shape[0],r_x)
V_x =  cp.random.rand(X1_cp.shape[1],r_x)
U_y = cp.random.rand(Y1_cp.shape[0],r_y)
A_fxy = cp.random.rand(r_x,r_y)
V_y = {}
for i in Y_cp.keys():
    V_y[i] = cp.random.rand(Y_cp[i].shape[1],r_y)

In [110]:
X1.sum().sum(), X1.sum().sum()/Y1.sum().sum(), X1.sum().sum()/Y2.sum().sum(), X1.sum().sum()/R_fxy.sum().sum()

(17938192.0, 0.057459348482602715, 0.0782136563858995, 2881.792811241691)

In [111]:
X1.sum().sum(), X1.sum().sum()/Y1.sum().sum(), X1.sum().sum()/Y2.sum().sum(), X1.sum().sum()/R_fxy.sum().sum()

(17938192.0, 0.057459348482602715, 0.0782136563858995, 2881.792811241691)

In [112]:
Lamb_inter = .00001 * X1.sum().sum()/R_fxy.sum().sum()
Lamb_Y = [.1* X1.sum().sum()/Y1.sum().sum(), .1* X1.sum().sum()/Y2.sum().sum()]
Lamb_sparsity =.0000000001
e = 0.000000001

In [113]:
U_x.shape, R_fxy_cp.shape, U_y.shape, Y1.shape, X1.shape
type(U_x), type(R_fxy_cp), type(U_y), type(Y1), type(X1)

(cupy.ndarray,
 cupy.ndarray,
 cupy.ndarray,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame)

In [114]:
Lamb_inter

0.028817928112416916

In [115]:
V_y[i].shape, U_y.shape

((7968, 40), (416, 40))

In [116]:
import random


In [117]:
import cupy as cp

# Get the number of available CUDA devices
device_count = cp.cuda.runtime.getDeviceCount()

if device_count > 0:
    print(f"Cupy is using {device_count} CUDA device(s).")
    for device_id in range(device_count):
        device_properties = cp.cuda.runtime.getDeviceProperties(device_id)
        print(f"Device {device_id}: {device_properties['name']}")
else:
    print("No CUDA devices found. Cupy is using the CPU.")

Cupy is using 1 CUDA device(s).
Device 0: b'NVIDIA RTX A4000'


In [118]:
A_fxy = cp.eye(A_fxy.shape[0])

In [119]:
epsilon = 0.000000001

In [123]:
Save_results = []
for iteration in range (0, 1500):
    aa = time.time()

    #.................................................
    U_x_batch = U_x.shape[0] // 1
    U_y_batch = U_y.shape[0] // 1
    V_batch = V_x.shape[0] //1
    U_x_list = list(range(U_x.shape[0]))
    U_y_list = list(range(U_y.shape[0]))
    V_list = list(range(V_x.shape[0]))
    U_x_sampling = random.sample(U_x_list, U_x_batch)
    U_y_sampling = random.sample(U_y_list, U_y_batch)
    V_sampling = random.sample(V_list, V_batch)
    
    
    U_x_sampled = U_x[U_x_sampling, :]
    U_y_sampled = U_y[U_y_sampling, :]
    R_fxy_cp_sampled = R_fxy_cp[U_x_sampling, :][:,U_y_sampling ]
    V_x_sampled = V_x[V_sampling, :]
    V_sampled = {}
    for i in V_y.keys():
        V_sampled[i] = V_y[i][V_sampling, :]
    X1_cp_sampled = X1_cp[U_x_sampling, ][:, V_sampling]
    #R_fxy_sampled = R_fxy_cp[U_x_Sampling, ][:, U_y_Sampling]
    Y_sampled = {}
    for i in V_y.keys():
        Y_sampled[i] = Y_cp[i][U_y_sampling, ][:,V_sampling ]

    #print('1', time.time() - aa)
    #................................................
    if iteration%50 == 0:
        print(iteration, U_y.dot(V_y[0].T).sum().sum())
        rmse_X1 = calculate_rmse(X1_cp_sampled.get(), U_x_sampled.get().dot(V_x_sampled.get().T))
        rmse_Y = []
        for i in V_y.keys():
            rec = U_y_sampled.dot(V_sampled[i].T)
            RM = calculate_rmse(Y_sampled[i].get(), rec.get())
            rmse_Y.append(RM)
        rmse_Relationship = calculate_rmse(R_fxy_cp_sampled.get(), U_x_sampled .get().dot(A_fxy.get()).dot(U_y_sampled.get().T))
        Save_results.append([iteration, rmse_X1, rmse_Y, rmse_Relationship])
        print(iteration, rmse_X1, rmse_Y, rmse_Relationship)
    #print('2', time.time() - aa)
    '''
    numerator_e = U_x_sampled.T.dot(R_fxy_cp_sampled.dot(U_y_sampled))+ 0.0000000001
    denominator_e = U_x_sampled.T.dot(U_x_sampled.dot(A_fxy.dot(U_y_sampled.T.dot(U_y_sampled)))) + Lamb_sparsity* A_fxy+ 0.0000000001
    second_term_e = (numerator_e)/(denominator_e)
    A_fxy = A_fxy * (second_term_e)

    type(A_fxy)
    '''
    #print('3', time.time() - aa)
    #Updating Ux
    numerator_f = (X1_cp_sampled.dot(V_x_sampled)) + (Lamb_inter * R_fxy_cp_sampled.dot(U_y_sampled.dot(A_fxy.T)))  + 0.0000000001
    denominator_f = (U_x_sampled.dot(V_x_sampled.T.dot(V_x_sampled))) + (Lamb_inter * U_x_sampled.dot(A_fxy.dot(U_y_sampled.T.dot(U_y_sampled.dot(A_fxy.T))))) + Lamb_sparsity* U_x_sampled + + 0.0000000001
    second_term_f = (numerator_f/denominator_f)
    U_x[U_x_sampling, ] = (U_x_sampled) * (second_term_f)

    #print('4', time.time() - aa)
    #Updating Vx
    
    numerator_c = (X1_cp_sampled.T.dot(U_x_sampled))  + 0.0000000001
    denominator_c = (V_x_sampled.dot(U_x_sampled.T.dot(U_x_sampled))) + Lamb_sparsity* V_x_sampled   + 0.0000000001
    second_term_c =  (numerator_c/denominator_c) 
    
    V_x[V_sampling, ] = (V_x_sampled) * (second_term_c)

    #print('5', time.time() - aa)
    
    for i in Y_cp.keys():
        if i == 0:
            numerator_g = Lamb_Y[i] * Y_sampled[i].dot(V_sampled[i]) + 0.0000000001
            denominator_g = Lamb_Y[i] * U_y_sampled.dot(V_sampled[i].T.dot(V_sampled[i])) + + 0.0000000001
            #print(U_y.dot(V_sampled[i].T.dot(V_sampled[i])).shape, type(denominator_g))
        if i>0:
            numerator_g = numerator_g + Lamb_Y[i] * Y_sampled[i].dot(V_sampled[i]) 
            #print(type(U_y.dot(V_sampled[i].T.dot(V_sampled[i]))), denominator_g.shape, U_y.dot(V_sampled[i].T.dot(V_sampled[i])).shape)
            denominator_g = denominator_g + Lamb_Y[i] * U_y_sampled.dot(V_sampled[i].T.dot(V_sampled[i]))
    
    numerator_g = numerator_g + Lamb_inter * R_fxy_cp_sampled.T.dot(U_x_sampled.dot(A_fxy))
    denominator_g = denominator_g + Lamb_inter * U_y_sampled.dot(A_fxy.T.dot(U_x_sampled.T.dot(U_x_sampled.dot(A_fxy)))) + Lamb_sparsity* U_y_sampled
            
    second_term_g = (numerator_g/denominator_g)
    
    U_y[U_y_sampling, ] = (U_y_sampled)* (second_term_g)


    #print('5', time.time() - aa)
    for i in V_y.keys():
        numerator_d = (Lamb_Y[i] * Y_sampled[i].T.dot(U_y_sampled))+ 0.0000000001
        denominator_d = (Lamb_Y[i] * V_sampled[i].dot(U_y_sampled.T.dot(U_y_sampled)))  + Lamb_sparsity* V_sampled[i]+ 0.0000000001
        second_term_d = (numerator_d/denominator_d)
        V_y[i][V_sampling, ] = (V_sampled[i])* (second_term_d)


    #print('7', time.time() - aa)

0 259836028.4164546
0 0.9631 [5.85] 2.673
50 259834264.0440308
50 0.9628 [5.846] 2.6544
100 259832547.2639077
100 0.9624 [5.8421] 2.6361
150 259831019.97215903
150 0.9621 [5.8384] 2.6183
200 259829924.9734289
200 0.9617 [5.8347] 2.6008
250 259828471.83815396
250 0.9613 [5.8311] 2.5836
300 259827128.2550925
300 0.961 [5.8277] 2.5668
350 259825849.3310663
350 0.9606 [5.8244] 2.5504
400 259824534.84241608
400 0.9602 [5.8211] 2.5342
450 259823192.81426784
450 0.9598 [5.8179] 2.5184
500 259821863.0495357
500 0.9595 [5.8149] 2.5029
550 259820717.30554682
550 0.9591 [5.8119] 2.4877
600 259819584.22954208
600 0.9587 [5.8089] 2.4727
650 259818132.92271227
650 0.9584 [5.8061] 2.4579
700 259816661.61491373
700 0.9581 [5.8033] 2.4435
750 259815225.94522017
750 0.9577 [5.8007] 2.4292
800 259813846.33263937
800 0.9574 [5.798] 2.4152
850 259812517.8758575
850 0.9571 [5.7955] 2.4014
900 259811178.78868192
900 0.9568 [5.793] 2.3879
950 259809839.73800084
950 0.9564 [5.7905] 2.3746
1000 259808461.688791

In [124]:
pd.DataFrame(U_x.get(), index = X1_training.columns).to_csv('U_x_JAIDR40M.csv')
pd.DataFrame(V_x.get(), index = X1_training.index).to_csv('V_x_JAIDR40M.csv')
pd.DataFrame(U_y.get(), index = Y1_training.columns).to_csv('U_y_JAIDR40M.csv')
pd.DataFrame(V_y[0].get(), index = Y1_training.index).to_csv('V_y1_JAIDR40M.csv')
#pd.DataFrame(V_y[1].get(), index = Y1_training.index).to_csv('V_y2_JAIDR40M.csv')
pd.DataFrame(A_fxy.get()).to_csv('Afxy_JAIDR20M.csv')


In [125]:
X1_training.to_csv(r'X1_training.csv')
Y1_training.to_csv(r'Y1_training.csv')
Y2_training.to_csv(r'Y2_training.csv')
X1_testing.to_csv(r'X1_testing.csv')
Y1_testing.to_csv(r'Y1_testing.csv')
Y2_testing.to_csv(r'Y2_testing.csv')